### 贝叶斯 垃圾邮件分类例子

In [4]:
# 1. create data
from numpy import *

def loadDataSet():
    postingList=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0, 1, 0, 1, 0, 1]    #1 is abusive, 0 not
    return postingList, classVec

In [2]:
#  2. creat vocablist
def createVocabList(dataSet):
    vocabSet = set([])  # create empty set
    for document in dataSet:
        vocabSet = vocabSet | set(document)  # union of the two sets
    return list(vocabSet)

In [3]:
# 3.ont hot type to convert inputset into 0,1 list
def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0] * len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else: 
            print("the word: %s is not in my Vocabulary!" % word)
    return returnVec

In [5]:
# 4. train bayes
def trainNB0(trainMatrix,trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory) / float(numTrainDocs)
    p0Num = ones(numWords)
    p1Num = ones(numWords)      # change to ones() 
    p0Denom = 2.0
    p1Denom = 2.0               # change to 2.0 to avoid zero
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = log(p1Num/p1Denom)          # change to log() to avoid mult to zero
    p0Vect = log(p0Num/p0Denom)          # change to log()
    return p0Vect, p1Vect, pAbusive

In [6]:
# 5.classify
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = sum(vec2Classify * p1Vec) + log(pClass1)    # element-wise mult
    p0 = sum(vec2Classify * p0Vec) + log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else: 
        return 0

In [7]:
# 6. vocab bag 
def bagOfWords2VecMN(vocabList, inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1
    return returnVec

In [8]:
# 7. train NB
def testingNB():
    listOPosts,listClasses = loadDataSet()
    myVocabList = createVocabList(listOPosts)
    trainMat=[]
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
    p0V, p1V, pAb = trainNB0(array(trainMat), array(listClasses))
    testEntry = ['love', 'my', 'dalmation']
    thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry,'classified as: ', classifyNB(thisDoc, p0V, p1V, pAb))
    testEntry = ['stupid', 'garbage']
    thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry,'classified as: ', classifyNB(thisDoc,p0V,p1V,pAb))


In [9]:
# 8. preprocess
def textParse(bigString):    # input is big string, output is word list
    import re
    listOfTokens = re.split(r'\W*', bigString)  # remove num, flag, and len <= 2
    return [tok.lower() for tok in listOfTokens if len(tok) > 2] 

In [31]:
# test
def spamTest():
    docList=[]
    classList = []
    fullText =[]
    for i in range(1,26):
        wordList = textParse(open('email/spam/%d.txt' % i, encoding='ISO-8859-1').read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1)
        wordList = textParse(open('email/ham/%d.txt' % i, encoding='ISO-8859-1').read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    vocabList = createVocabList(docList)  # create vocabulary
    trainingSet = range(50)
    testSet=[]           # create test set
    for i in range(10):
        randIndex = int(random.uniform(0, len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        # del(trainingSet[randIndex])  
    trainMat=[]
    trainClasses = []
    for docIndex in trainingSet:  # train the classifier (get probs) trainNB0
        trainMat.append(bagOfWords2VecMN(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V,p1V,pSpam = trainNB0(array(trainMat),array(trainClasses))
    errorCount = 0
    for docIndex in testSet:  # classify the remaining items
        wordVector = bagOfWords2VecMN(vocabList, docList[docIndex])
        if classifyNB(array(wordVector), p0V, p1V, pSpam) != classList[docIndex]:
            errorCount += 1
            print("classification error", docList[docIndex])
    print('the error rate is: ', float(errorCount)/len(testSet))

In [41]:
# find most show 30 words
def calcMostFreq(vocabList,fullText):
    import operator
    freqDict = {}
    for token in vocabList:
        freqDict[token]=fullText.count(token)
    sortedFreq = sorted(freqDict.items(), key=operator.itemgetter(1), reverse=True) 
    return sortedFreq[:30]  

In [46]:
def localWords(feed1,feed0):
    import feedparser
    docList=[]
    classList = []
    fullText =[]
    minLen = min(len(feed1['entries']), len(feed0['entries']))
    for i in range(minLen):
        wordList = textParse(feed1['entries'][i]['summary'])
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1)  # NY is class 1
        wordList = textParse(feed0['entries'][i]['summary'])
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    vocabList = createVocabList(docList)  # create vocabulary
    top30Words = calcMostFreq(vocabList,fullText)   # remove top 30 words
    for pairW in top30Words:
        if pairW[0] in vocabList: vocabList.remove(pairW[0])
    trainingSet = range(2*minLen)
    testSet=[]           # create test set
    for i in range(10):
        randIndex = int(random.uniform(0, len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del(trainingSet[randIndex])  
    trainMat=[]; trainClasses = []
    for docIndex in trainingSet:#train the classifier (get probs) trainNB0
        trainMat.append(bagOfWords2VecMN(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V,p1V,pSpam = trainNB0(array(trainMat),array(trainClasses))
    errorCount = 0
    for docIndex in testSet:        #classify the remaining items
        wordVector = bagOfWords2VecMN(vocabList, docList[docIndex])
        if classifyNB(array(wordVector),p0V,p1V,pSpam) != classList[docIndex]:
            errorCount += 1
    print('the error rate is: ',float(errorCount)/len(testSet))
    return vocabList, p0V, p1V

In [14]:
# get top words by p > e^-6
def getTopWords(ny,sf):
    import operator
    vocabList,p0V,p1V=localWords(ny,sf)
    topNY=[]; topSF=[]
    for i in range(len(p0V)):
        if p0V[i] > -6.0 : topSF.append((vocabList[i],p0V[i]))
        if p1V[i] > -6.0 : topNY.append((vocabList[i],p1V[i]))
    sortedSF = sorted(topSF, key=lambda pair: pair[1], reverse=True)
    print("SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**")
    for item in sortedSF:
        print(item[0])
    sortedNY = sorted(topNY, key=lambda pair: pair[1], reverse=True)
    print("NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**")
    for item in sortedNY:
        print(item[0])

### 进行上述测试

In [18]:
listOfPosts, listClass = loadDataSet()
print(listOfPosts, listClass)

[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'], ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'], ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'], ['stop', 'posting', 'stupid', 'worthless', 'garbage'], ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'], ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']] [0, 1, 0, 1, 0, 1]


In [20]:
myVocabList = createVocabList(listOfPosts)
print(myVocabList)

['dalmation', 'problems', 'to', 'help', 'flea', 'stupid', 'take', 'love', 'please', 'him', 'posting', 'steak', 'quit', 'so', 'dog', 'cute', 'buying', 'worthless', 'mr', 'park', 'licks', 'my', 'stop', 'ate', 'is', 'not', 'I', 'has', 'maybe', 'garbage', 'how', 'food']


In [21]:
setOfWords2Vec(myVocabList, listOfPosts[0])

[0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0]

In [22]:
setOfWords2Vec(myVocabList, listOfPosts[3])

[0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0]

In [25]:
trainMat = []
for postinDoc in listOfPosts:
    trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
p0v, p1v, pAb = trainNB0(trainMat, listClass)
print(p0v, p1v, pAb)

[-2.56494936 -2.56494936 -2.56494936 -2.56494936 -2.56494936 -3.25809654
 -3.25809654 -2.56494936 -2.56494936 -2.15948425 -3.25809654 -2.56494936
 -3.25809654 -2.56494936 -2.56494936 -2.56494936 -3.25809654 -3.25809654
 -2.56494936 -3.25809654 -2.56494936 -1.87180218 -2.56494936 -2.56494936
 -2.56494936 -3.25809654 -2.56494936 -2.56494936 -3.25809654 -3.25809654
 -2.56494936 -3.25809654] [-3.04452244 -3.04452244 -2.35137526 -3.04452244 -3.04452244 -1.65822808
 -2.35137526 -3.04452244 -3.04452244 -2.35137526 -2.35137526 -3.04452244
 -2.35137526 -3.04452244 -1.94591015 -3.04452244 -2.35137526 -1.94591015
 -3.04452244 -2.35137526 -3.04452244 -3.04452244 -2.35137526 -3.04452244
 -3.04452244 -2.35137526 -3.04452244 -3.04452244 -2.35137526 -2.35137526
 -3.04452244 -2.35137526] 0.5


In [27]:
testingNB()

['love', 'my', 'dalmation'] classified as:  0
['stupid', 'garbage'] classified as:  1


In [36]:
spamTest()

classification error []
classification error []
classification error []
the error rate is:  0.3


In [47]:
import feedparser
ny = feedparser.parse('http://newyork.craigslist.org/stp/index.rss')
print(ny)
sf = feedparser.parse('http://sfbay.craigslist.org/stp/index.rss')
print(sf)
vocablist, pSF, pNY = localWords(ny, sf)
print(vocablist, pSF, pNY)

{'feed': {'html': {'class': 'no-js'}, 'links': [{'type': 'text/css', 'rel': 'stylesheet', 'media': 'all', 'href': 'https://www.craigslist.org/styles/simple-page.css?v=5d2eef2ed3ec646af97d5d471601b594'}, {'type': 'text/css', 'rel': 'stylesheet', 'media': 'all', 'href': 'https://www.craigslist.org/styles/jquery-ui-clcustom.css?v=3b05ddffb7c7f5b62066deff2dda9339'}, {'type': 'text/css', 'rel': 'stylesheet', 'media': 'all', 'href': 'https://www.craigslist.org/styles/jquery.qtip-2.2.1.css?v=cd202aead4d1dd4894fbae4ade23fcf8'}], 'meta': {'name': 'viewport', 'content': 'width=device-width,initial-scale=1'}}, 'entries': [], 'bozo': 1, 'headers': {'Connection': 'keep-alive', 'Last-Modified': 'Fri, 31 Jan 2020 16:26:43 GMT', 'Date': 'Fri, 31 Jan 2020 16:26:43 GMT', 'Content-Encoding': 'gzip', 'Vary': 'Accept-Encoding', 'Content-Length': '2071', 'X-Frame-Options': 'SAMEORIGIN', 'Content-Type': 'text/html; charset=UTF-8', 'Set-Cookie': 'cl_b=4|5eed23421a3f0436d2c72f354619a86dc0f57e18|1580488003aRpNE

IndexError: range object index out of range

In [48]:
getTopWords(ny, sf)

IndexError: range object index out of range